In [1]:
import pandas as pd

# 데이터 집계하는 범위 
date_range = pd.date_range(start='2018-01-01', end='2022-06-30', freq='H')

df_all = pd.DataFrame(date_range, columns=['date'])

df_all['Year'] = df_all['date'].dt.year
df_all['Month'] = df_all['date'].dt.month
df_all['Day'] = df_all['date'].dt.day
df_all['UTCHour'] = df_all['date'].dt.hour
df_all = df_all.drop('date', axis=1)

df_all.set_index(['Year','Month','Day','UTCHour'])

df_all.head()

,Year,Month,Day,UTCHour
0,2018,1,1,0
1,2018,1,1,1
2,2018,1,1,2
3,2018,1,1,3
4,2018,1,1,4


## 서울, 베이징 미세먼지 데이터

In [2]:
df_seoul_pm_2008_2011 = pd.read_csv('./rawfiles/seoul-pm/2008-2011.csv', encoding='cp949')
df_seoul_pm_2012_2015 = pd.read_csv('./rawfiles/seoul-pm/2012-2015.csv', encoding='cp949')
df_seoul_pm_2016_2019 = pd.read_csv('./rawfiles/seoul-pm/2016-2019.csv', encoding='cp949')
df_seoul_pm_2020_2021 = pd.read_csv('./rawfiles/seoul-pm/2020-2021.csv', encoding='cp949')
df_seoul_pm_2022 = pd.read_csv('./rawfiles/seoul-pm/2022.csv', encoding='cp949')

df_seoul_pm = pd.concat([
    df_seoul_pm_2008_2011, df_seoul_pm_2012_2015, df_seoul_pm_2016_2019, df_seoul_pm_2020_2021, df_seoul_pm_2022
], axis=0)

df_seoul_pm = df_seoul_pm.groupby('일시').first() # 지역별 평균 값만 담는다.

In [3]:
df_seoul_pm['Year'] = df_seoul_pm.index.str[:4].astype('int')
df_seoul_pm['Month'] = df_seoul_pm.index.str[5:7].astype('int')
df_seoul_pm['Day'] = df_seoul_pm.index.str[8:10].astype('int')
df_seoul_pm['UTCHour'] = df_seoul_pm.index.str.extract(r' (\d+):', expand=False).astype('int')

In [4]:
df_seoul_pm = df_seoul_pm.drop('구분', axis=1)

In [5]:
df_seoul_pm['Seoul_PM2.5'] = df_seoul_pm['초미세먼지(PM2.5)'].combine_first(df_seoul_pm['초미세먼지(PM25)'])
df_seoul_pm = df_seoul_pm.drop('초미세먼지(PM2.5)', axis=1)
df_seoul_pm = df_seoul_pm.drop('초미세먼지(PM25)', axis=1)
df_seoul_pm = df_seoul_pm.rename(columns={'미세먼지(PM10)': 'Seoul_PM10'})

In [6]:
df_seoul_pm

,Seoul_PM10,Year,Month,Day,UTCHour,Seoul_PM2.5
일시,,,,,,
2008-01-01 10:00,30.0,2008,1,1,10,11.0
2008-01-01 11:00,29.0,2008,1,1,11,13.0
2008-01-01 12:00,29.0,2008,1,1,12,12.0
2008-01-01 13:00,28.0,2008,1,1,13,12.0
2008-01-01 14:00,27.0,2008,1,1,14,13.0
...,...,...,...,...,...,...
2022-12-31 5:00,33.0,2022,12,31,5,25.0
2022-12-31 6:00,34.0,2022,12,31,6,25.0
2022-12-31 7:00,34.0,2022,12,31,7,26.0


In [7]:
# 베이징 미세먼지 데이터
df_beijing_pm25 = pd.read_csv('./rawfiles/beijing-pm.txt', sep='\s+')
df_beijing_pm25 = df_beijing_pm25.drop(['Retrospective', 'PM10_mask'], axis=1)
df_beijing_pm25 = df_beijing_pm25.rename(columns={'PM2.5': 'Beijing_PM2.5'})
df_beijing_pm25 = df_beijing_pm25.groupby(['Year','Month','Day','UTCHour']).mean()

df_beijing_pm10 = pd.read_csv('https://raw.githubusercontent.com/SeojinSeojin/data-storage/main/all_air.csv')
df_beijing_pm10 = df_beijing_pm10.replace(-9999, float("nan"))
df_beijing_pm10 = df_beijing_pm10.rename(columns={'year':'Year', 'month':'Month', 'date':'Day', 'hour':'UTCHour', 'PM10': 'Beijing_PM10'})

In [8]:
df_beijing_pm10 = df_beijing_pm10.loc[ : ,['Year','Month','Day','UTCHour','Beijing_PM10']]

df_beijing_pm = pd.merge(df_beijing_pm25, df_beijing_pm10, on=['Year', 'Month', 'Day', 'UTCHour'])

# 위의 df_all로 left outer join
# left join을 하는 이유: 2016년 12월 10일 1시 데이터가 df_seoul_pm에 없더라도, 모두 NaN으로 입력될 수 있도록
df = pd.merge(df_all, df_seoul_pm, on=['Year', 'Month', 'Day', 'UTCHour'], how='left')
df = pd.merge(df, df_beijing_pm, on=['Year', 'Month', 'Day', 'UTCHour'], how='left')

df[['Seoul_PM2.5', 'Seoul_PM10', 'Beijing_PM2.5', 'Beijing_PM10']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Seoul_PM2.5,39385.0,21.885718,16.906571,1.00,11.00,18.00,28.0,162.00
Seoul_PM10,39385.0,38.494401,30.767995,3.00,21.00,33.00,48.0,868.00
Beijing_PM2.5,36897.0,35.400181,31.434107,2.07,13.39,26.07,45.6,340.52
Beijing_PM10,35642.0,67.376354,89.626953,1.00,29.00,53.00,85.0,6450.00


In [9]:
df

,Year,Month,Day,UTCHour,Seoul_PM10,Seoul_PM2.5,Beijing_PM2.5,Beijing_PM10
0,2018,1,1,0,36.0,18.0,25.625,102.0
1,2018,1,1,1,36.0,18.0,33.540,95.0
2,2018,1,1,2,35.0,17.0,45.430,NaN
3,2018,1,1,3,34.0,18.0,58.410,NaN
4,2018,1,1,4,34.0,18.0,68.650,NaN
...,...,...,...,...,...,...,...,...
39380,2022,6,29,20,12.0,6.0,10.010,16.0
39381,2022,6,29,21,9.0,4.0,9.990,18.0
39382,2022,6,29,22,6.0,2.0,9.860,19.0
39383,2022,6,29,23,7.0,3.0,10.500,20.0


In [10]:
# 결측치 값 개수 출력

print('# of null Seoul pm2.5 values : ', df['Seoul_PM2.5'].isna().sum())
print('# of null Seoul pm10 values : ', df['Seoul_PM10'].isna().sum())
print('# of null Beijing pm2.5 values : ', df['Beijing_PM2.5'].isna().sum())
print('# of null Beijing pm10 values : ', df['Beijing_PM10'].isna().sum())

# of null Seoul pm2.5 values :  0
# of null Seoul pm10 values :  0
# of null Beijing pm2.5 values :  2488
# of null Beijing pm10 values :  3743


## 서울 대기질 데이터

In [11]:
sw_2014 = pd.read_excel("./rawfiles/seoul-weather/2014.xlsx")
sw_2015 = pd.read_excel("./rawfiles/seoul-weather/2015.xlsx")
sw_2016 = pd.read_excel("./rawfiles/seoul-weather/2016.xlsx")
sw_2017 = pd.read_excel("./rawfiles/seoul-weather/2017.xlsx")
sw_2018 = pd.read_csv("./rawfiles/seoul-weather/2018.csv", encoding='cp949')
sw_2019 = pd.read_csv("./rawfiles/seoul-weather/2019.csv", encoding='cp949')
sw_2020 = pd.read_csv("./rawfiles/seoul-weather/2020.csv", encoding='cp949')
sw_2021 = pd.read_csv("./rawfiles/seoul-weather/2021.csv", encoding='cp949')
sw_2022 = pd.read_csv("./rawfiles/seoul-weather/2022.csv", encoding='cp949')

sw_all = pd.concat([sw_2014, sw_2015, sw_2016, sw_2017, sw_2018, sw_2019, sw_2020, sw_2021, sw_2022], axis=0)
sw_all['Seoul_SO2'] = sw_all.apply(lambda row: row['아황산가스농도(ppm)'] if pd.notna(row['아황산가스농도(ppm)']) else row['아황산가스(ppm)'], axis=1)
sw_all = sw_all.groupby('측정일시').mean()

In [12]:

sw_all = sw_all.rename(columns={'이산화질소농도(ppm)':'Seoul_NO2', 
                                '오존농도(ppm)':'Seoul_O3', 
                                '일산화탄소농도(ppm)':'Seoul_CO',
                                })
sw_all = sw_all.drop(['미세먼지농도(㎍/㎥)','초미세먼지농도(㎍/㎥)','미세먼지(㎍/㎥)','초미세먼지(㎍/㎥)', '아황산가스농도(ppm)', '아황산가스(ppm)','이산화탄소농도(ppm)'], axis=1)
sw_all.head()

,Seoul_NO2,Seoul_O3,Seoul_CO,Seoul_SO2
측정일시,,,,
20140101,0.035949,0.011175,NaN,0.009725
20140102,0.043100,0.008425,NaN,0.007950
20140103,0.059225,0.003250,NaN,0.008550
20140104,0.038500,0.007425,NaN,0.006275
20140105,0.040500,0.006385,NaN,0.006000


In [13]:
sw_all.tail()

,Seoul_NO2,Seoul_O3,Seoul_CO,Seoul_SO2
측정일시,,,,
20221227,0.04670,0.00826,0.872,0.00366
20221228,0.02792,0.01738,0.564,0.00352
20221229,0.03286,0.01290,0.576,0.00380
20221230,0.03366,0.01472,0.606,0.00360
20221231,0.04198,0.00830,0.786,0.00378


In [14]:
sw_all.index = sw_all.index.astype(str)

sw_all['Year'] = pd.to_datetime(sw_all.index.str[:4] + '-' + sw_all.index.str[4:6] + '-' + sw_all.index.str[6:]).year
sw_all['Month'] = pd.to_datetime(sw_all.index.str[:4] + '-' + sw_all.index.str[4:6] + '-' + sw_all.index.str[6:]).month
sw_all['Day'] = pd.to_datetime(sw_all.index.str[:4] + '-' + sw_all.index.str[4:6] + '-' + sw_all.index.str[6:]).day
sw_all['UTCHour'] = 0 # 일별 단위이므로 시간은 일단 0으로 밀어넣는다.

In [15]:
sw_all.tail()

,Seoul_NO2,Seoul_O3,Seoul_CO,Seoul_SO2,Year,Month,Day,UTCHour
측정일시,,,,,,,,
20221227,0.04670,0.00826,0.872,0.00366,2022,12,27,0
20221228,0.02792,0.01738,0.564,0.00352,2022,12,28,0
20221229,0.03286,0.01290,0.576,0.00380,2022,12,29,0
20221230,0.03366,0.01472,0.606,0.00360,2022,12,30,0
20221231,0.04198,0.00830,0.786,0.00378,2022,12,31,0


In [16]:
df = pd.merge(df, sw_all, on=['Year', 'Month', 'Day', 'UTCHour'], how='left')

In [17]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Year,39385.0,2019.774584,1.312477,2018.000000,2019.000000,2020.000000,2021.000000,2022.000000
Month,39385.0,6.190733,3.435601,1.000000,3.000000,6.000000,9.000000,12.000000
Day,39385.0,15.707249,8.788965,1.000000,8.000000,16.000000,23.000000,31.000000
UTCHour,39385.0,11.499708,6.922429,0.000000,5.000000,11.000000,17.000000,23.000000
Seoul_PM10,39385.0,38.494401,30.767995,3.000000,21.000000,33.000000,48.000000,868.000000
Seoul_PM2.5,39385.0,21.885718,16.906571,1.000000,11.000000,18.000000,28.000000,162.000000
Beijing_PM2.5,36897.0,35.400181,31.434107,2.070000,13.390000,26.070000,45.600000,340.520000
Beijing_PM10,35642.0,67.376354,89.626953,1.000000,29.000000,53.000000,85.000000,6450.000000
Seoul_NO2,1642.0,0.028310,0.011341,0.007840,0.019549,0.026323,0.035698,0.078500
Seoul_O3,1642.0,0.024365,0.011649,0.002974,0.015465,0.023396,0.031975,0.073740


NO2, O3, CO, SO2는 일별로 기록되므로 count가 적다.

* 이들을 전부 앞 값으로 채우거나 (fillna(method='forward')) 
* 전날 값과 다음날 값의 평균으로 채우는 등의 방법을 사용해서 채워서 사용할 수 있다.

## 화력 발전량 데이터

In [18]:
thermalPower = pd.read_csv("./rawfiles/korea-thermal-power.csv", encoding='cp949')

thermalPower.head(35)

,거래일자,지역,연료원,발전량(MWh)
0,2018-01-01,강원도,LNG,7003
1,2018-01-01,강원도,석탄,46702
2,2018-01-01,경기도,LNG,114569
3,2018-01-01,경기도,석탄,1095
4,2018-01-01,경기도,유류,722
5,2018-01-01,경상남도,석탄,143844
6,2018-01-01,경상남도,유류,183
7,2018-01-01,경상북도,LNG,0
8,2018-01-01,경상북도,석탄,2742
9,2018-01-01,광주시,LNG,1784


지역별 발전량으로 합친다.

| 일자 | 강원도_LNG | 강원도_석탄 | 경기도_LNG | 경기도_석탄 | 경기도_유류 | 경상북도_LNG | 경상북도_석탄 | 광주_LNG | 대구_LNG | 대구_석탄 | 대구_유류 | 대전_LNG | 대전_유류 | 부산_LNG | 부산_석탄 | 서울_LNG | 세종_LNG | 울산_LNG | 울산_유류 | 인천_LNG | 인천_석탄 | 전라남도_LNG | 전라남도_석탄 | 전라북도_LNG | 전라북도_석탄 | 전라북도_유류 | 제주도_유류 | 충청남도_LNG | 충청남도_석탄 | 충청남도_유류 | 충청북도_유류 |
| -- | -- | -- | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| date | GW_LNG | GW_Coal | GG_LNG | GG_Coal | GG_유류 | GB_LNG | GB_Coal | Gwangju_LNG | Daegu_LNG | Daegu_Coal | Daegu_Oil | Daejeon_LNG | Daejeon_Oil | Busan_LNG | Busan_Coal | Seoul_LNG | Sejong_LNG | Ulsan_LNG | Ulsan_Oil | Incheon_LNG | Incheon_Coal | JN_LNG | JN_Coal | JB_LNG | JB_Coal | JB_Oil | Jeju_Oil | CN_LNG | CN_Coal | CN_Oil | CB_Oil |

In [19]:
date_range = pd.date_range(start='2018-01-01', end='2022-06-30', freq='D')

th_data_all = pd.DataFrame(date_range, columns=['date'])

dict_location = {'강원도':'GW', '경기도':'GG', '경상북도':'GB','경상남도':'GN', '광주시':'Gwangju', '대구시':'Daegu','대전시':'Daejeon','부산시':'Busan','서울시':'Seoul','세종시':'Sejong','울산시':'Ulsan','인천시':'Incheon','전라남도':'JN','전라북도':'JB','제주도':'Jeju','충청남도':'CN','충청북도':'CB' }
dict_material = {'LNG':'LNG', '석탄':'Coal','유류':'Oil'}

thermalPower = thermalPower.rename(columns={' 발전량(MWh) ':'power'})

dict_all = dict()

for index, row in thermalPower.iterrows():
    time = row['거래일자']
    location = row['지역']
    material = row['연료원']
    power = row['power']
    key = dict_location[location] + "_" + dict_material[material]
    if time in dict_all.keys():
        prevValue = dict_all[time]
        prevValue[key] = power
    else:
        v = dict()
        v[key] = power
        dict_all[time] = v

In [20]:
thermal_power_pd = pd.DataFrame.from_dict(dict_all, orient='index')
thermal_power_pd['date'] = pd.to_datetime(thermal_power_pd.index)
th_data_all['date'] = th_data_all['date']

In [21]:
th_data_all = pd.merge(th_data_all, thermal_power_pd, on=['date'], how='left') 
th_data_all.head()

,date,GW_LNG,GW_Coal,GG_LNG,GG_Coal,GG_Oil,GN_Coal,GN_Oil,GB_LNG,GB_Coal,...,JN_Coal,JB_LNG,JB_Coal,JB_Oil,Jeju_Oil,CN_LNG,CN_Coal,CN_Oil,CB_Oil,Jeju_LNG
0,2018-01-01,7003,46702,114569,1095,722,143844,183,0,2742,...,26116,0,19139,404,6834,10867,290788,0,1107,NaN
1,2018-01-02,15290,38744,220946,1303,15178,159881,275,4481,2683,...,28611,7107,19027,402,8006,21904,334588,0,721,NaN
2,2018-01-03,17401,39059,237802,1404,13141,164212,349,8662,2632,...,31074,7873,19035,397,6160,28111,345349,0,322,NaN
3,2018-01-04,17937,39043,248705,1410,17898,164620,288,9033,2636,...,30976,10538,19094,393,7588,32898,340057,0,1128,NaN
4,2018-01-05,16957,39233,241342,1406,14630,164327,323,8518,2649,...,31193,10080,19138,402,7255,27683,348762,0,1153,NaN


In [22]:
th_data_all['Year'] = pd.to_datetime(th_data_all['date']).dt.year
th_data_all['Month'] = pd.to_datetime(th_data_all['date']).dt.month
th_data_all['Day'] = pd.to_datetime(th_data_all['date']).dt.day
th_data_all['UTCHour'] = 0 # 일별 단위이므로 시간은 일단 0으로 밀어넣는다.

In [23]:
th_data_all = th_data_all.drop('date', axis=1)

In [24]:
df = pd.merge(df, th_data_all, on=['Year', 'Month', 'Day', 'UTCHour'], how='left')

In [25]:
df.head(25)

,Year,Month,Day,UTCHour,Seoul_PM10,Seoul_PM2.5,Beijing_PM2.5,Beijing_PM10,Seoul_NO2,Seoul_O3,...,JN_Coal,JB_LNG,JB_Coal,JB_Oil,Jeju_Oil,CN_LNG,CN_Coal,CN_Oil,CB_Oil,Jeju_LNG
0,2018,1,1,0,36.0,18.0,25.625,102.0,0.035946,0.010081,...,26116.0,0.0,19139.0,404.0,6834.0,10867.0,290788.0,0.0,1107.0,NaN
1,2018,1,1,1,36.0,18.0,33.540,95.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,1,1,2,35.0,17.0,45.430,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,1,1,3,34.0,18.0,58.410,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,1,1,4,34.0,18.0,68.650,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018,1,1,5,34.0,17.0,66.520,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2018,1,1,6,34.0,19.0,59.720,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2018,1,1,7,35.0,19.0,60.070,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018,1,1,8,35.0,19.0,59.300,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2018,1,1,9,35.0,19.0,55.480,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 모두 합쳐진 csv 파일로 내보내기

In [26]:
df.to_csv('all.csv', index=False)